# Let's import a bunch of stuff

In [1]:
import sys
import seaborn as sb
import scanpy.api as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import preprocessing
from sklearn.preprocessing import MaxAbsScaler
import argparse
import logging
import gc
from skbio.stats.composition import clr
%matplotlib inline
# Savefile path
savepath = 'V6.7.flareAb.h5ad'
saveno_norm_path = 'FlareDataset.h5ad'
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

# Import data file

In [2]:
df = pd.read_excel('flare.covars.again.xlsx')
df.head(10)

,RUN_WELL,ID,ALIAS,SOURCE,STATUS,TR/UNT,PAIR
0,1_1,0_904194200_904194200,"FLARE2,3,or8",UCSF,FLARE,"Unknown (2 TR, 1 UNT)",True
1,1_1,0_1716_1716,-,-,-,-,-
2,1_1,0_IGTB256,IGTB256,IMMVAR,HEALTHY,-,-
3,1_1,0_1892_1892,-,-,-,-,-
4,1_1,0_900805200_900805200,1804_1804,CLUES,HEALTHY,-,-
5,1_1,0_904236200_904236200,-,-,-,-,-
6,1_1,0_Criswell_Ye_E04_1941DNA01_Criswell_Ye_E04_19...,-,-,-,-,-
7,1_1,0_1754_1754,1754_1754,CLUES,HEALTHY,-,-
8,1_1,0_IGTB1921,IGTB1921,IMMVAR,HEALTHY,-,-
9,1_1,0_1763_1763,"FLARE2,3,or8",UCSF,FLARE,"Unknown (2 TR, 1 UNT)",True


# Combine flare samples

In [3]:
# Flare cohort files
files = [
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/1_1.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/1_2.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/2_1.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/2_2.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/3_1.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/3_2.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/3_3.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/3_4.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/4_1.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/4_2.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/4_3.adt.mrna.demuxed.freemuxed.h5ad',
'/ye/yelabstore2/george/Sasha_Flare_Studies/10xcount/combined.h5ads/again/4_4.adt.mrna.demuxed.freemuxed.h5ad']
flaregps = [
'1_1', '1_2', '2_1', '2_2',
'3_1', '3_2', '3_3', '3_4',
'4_1', '4_2', '4_3', '4_4']
DOUBLETS_dict = dict()
# Concatenate files
for ii in range(len(files)):
    df = pd.read_excel('flare.covars.again.xlsx')
    # This is crucial because Untreated and Treated were run in different runs and this is the only way to map it back.
    df = df[df['RUN_WELL'] == flaregps[ii]]
    if ii ==0:
        adata = sc.read(files[ii])
        print('Initial')
        print(adata)
        # Keep only singlets
        doubletlist2 = [adata.obs['BEST'][i].split('-')[0] for i in range(len(adata.obs['BEST']))]
        adata.obs['DOUBLETS'] = doubletlist2
        adata = adata[adata.obs['DOUBLETS']=='SNG']
        print('SNG confirmation via SNG- tag in BEST')

        print(adata)
        inds0 = np.unique(adata.obs['BEST'].tolist())
        inds =[]
        for i in range(len(inds0)):
            inds.append('-'.join(inds0[i].split('-')[1:]))
        inds = pd.Series(inds)
        inds0 = pd.Series(inds0)
        booleanlist = inds0[inds.isin(np.unique(df['ID'].tolist())).tolist()]
        print('Removing barcodes with no covariate data')
        adata = adata[adata.obs['BEST'].isin(booleanlist)]
        print(adata)
        
        inds0 = np.unique(adata.obs['BEST'].tolist())
        inds = []
        for i in range(len(inds0)):
            inds.append('-'.join(inds0[i].split('-')[1:]))
        
        # Preallocate space
        adata.obs['BEST'] = adata.obs['BEST'].astype('object')
        adata.obs['SOURCE'] = adata.obs['BEST']
        adata.obs['STATUS'] = adata.obs['BEST']
        adata.obs['TR_UNT'] = adata.obs['BEST']
        adata.obs['ind_cov'] = adata.obs['BEST']
        for xcv in range(len(inds)):
            adata.obs['SOURCE'][adata.obs['BEST']==inds0[xcv]] = df['SOURCE'][df['ID'].isin([inds[xcv]])].values[0]
            adata.obs['STATUS'][adata.obs['BEST']==inds0[xcv]] = df['STATUS'][df['ID'].isin([inds[xcv]])].values[0]
            adata.obs['TR_UNT'][adata.obs['BEST']==inds0[xcv]] = df['TR/UNT'][df['ID'].isin([inds[xcv]])].values[0]
            if df['ALIAS'][df['ID'].isin([inds[xcv]])].values[0] == 'FLARE2,3,or8':
                adata.obs['ind_cov'][adata.obs['BEST']==inds0[xcv]] = df['ID'][df['ID'].isin([inds[xcv]])].values[0].split(',')[0]
            else:
                adata.obs['ind_cov'][adata.obs['BEST']==inds0[xcv]] = df['ALIAS'][df['ID'].isin([inds[xcv]])].values[0]
           
        del adata.obs['N.SNP']
        del adata.obs['BEST']
        del adata.obs['mUMI']
        del adata.obs['aUMI']
        del adata.obs['NUM.SNPS']
        del adata.obs['DROPLET.TYPE']
        del adata.obs['BEST.GUESS']
        del adata.obs['DOUBLETS']
        print(str('Structure details: ' + str(adata)))
        adata.obs['well'] = np.repeat(flaregps[ii], len(adata.obs_names))
        adata.obs['batch_cov'] = np.repeat(flaregps[ii][:-2], len(adata.obs_names))
    else:
        bdata = sc.read(files[ii])
        print('Initial')
        print(bdata)
        # Keep only singlets
        doubletlist2 = [bdata.obs['BEST'][i].split('-')[0] for i in range(len(bdata.obs['BEST']))]
        bdata.obs['DOUBLETS'] = doubletlist2
        bdata = bdata[bdata.obs['DOUBLETS']=='SNG']
        print('SNG confirmation via SNG- tag in BEST')
        
        print(bdata)
        inds0 = np.unique(bdata.obs['BEST'].tolist())
        inds = []
        for i in range(len(inds0)):
            inds.append('-'.join(inds0[i].split('-')[1:]))
        inds = pd.Series(inds)
        inds0 = pd.Series(inds0)
        booleanlist = inds0[inds.isin(np.unique(df['ID'].tolist())).tolist()]
        print('Removing barcodes with no covariate data')
        bdata = bdata[bdata.obs['BEST'].isin(booleanlist)]
        print(bdata)
        
        inds0 = np.unique(bdata.obs['BEST'].tolist())
        inds = []
        for i in range(len(inds0)):
            inds.append('-'.join(inds0[i].split('-')[1:]))
        
        # Preallocate space
        bdata.obs['BEST'] = bdata.obs['BEST'].astype('object')
        bdata.obs['SOURCE'] = bdata.obs['BEST']
        bdata.obs['STATUS'] = bdata.obs['BEST']
        bdata.obs['TR_UNT'] = bdata.obs['BEST']
        bdata.obs['ind_cov'] = bdata.obs['BEST']
        for xcv in range(len(inds)):
            bdata.obs['SOURCE'][bdata.obs['BEST']==inds0[xcv]] = df['SOURCE'][df['ID'].isin([inds[xcv]])].values[0]
            bdata.obs['STATUS'][bdata.obs['BEST']==inds0[xcv]] = df['STATUS'][df['ID'].isin([inds[xcv]])].values[0]
            bdata.obs['TR_UNT'][bdata.obs['BEST']==inds0[xcv]] = df['TR/UNT'][df['ID'].isin([inds[xcv]])].values[0]
            if df['ALIAS'][df['ID'].isin([inds[xcv]])].values[0] == 'FLARE2,3,or8':
                bdata.obs['ind_cov'][bdata.obs['BEST']==inds0[xcv]] = df['ID'][df['ID'].isin([inds[xcv]])].values[0].split(',')[0]
            else:    
                bdata.obs['ind_cov'][bdata.obs['BEST']==inds0[xcv]] = df['ALIAS'][df['ID'].isin([inds[xcv]])].values[0]
                               
        del bdata.obs['N.SNP']
        del bdata.obs['BEST']
        del bdata.obs['mUMI']
        del bdata.obs['aUMI']
        del bdata.obs['NUM.SNPS']
        del bdata.obs['DROPLET.TYPE']
        del bdata.obs['BEST.GUESS']
        del bdata.obs['DOUBLETS']
        bdata.obs['well'] = np.repeat(flaregps[ii], len(bdata.obs_names))
        bdata.obs['batch_cov'] = np.repeat(flaregps[ii][:-2], len(bdata.obs_names))
        # Concatenate files.
        adata = adata.concatenate(bdata)
        print(str('Structure details: ' + str(adata)))

Only considering the two last: ['.freemuxed', '.h5ad'].
Only considering the two last: ['.freemuxed', '.h5ad'].
Initial
AnnData object with n_obs × n_vars = 12401 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS'
    var: 'gene_ids', 'protein?'
SNG confirmation via SNG- tag in BEST
View of AnnData object with n_obs × n_vars = 8948 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS', 'DOUBLETS'
    var: 'gene_ids', 'protein?'
Removing barcodes with no covariate data
View of AnnData object with n_obs × n_vars = 8948 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS', 'DOUBLETS'
    var: 'gene_ids', 'protein?'


/netapp/home/richardkperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/netapp/home/richardkperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/netapp/home/richardkperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/netapp/home/richardkperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.p

Structure details: AnnData object with n_obs × n_vars = 8948 × 32758 
    obs: 'SOURCE', 'STATUS', 'TR_UNT', 'ind_cov'
    var: 'gene_ids', 'protein?'
Only considering the two last: ['.freemuxed', '.h5ad'].
Only considering the two last: ['.freemuxed', '.h5ad'].
Initial
AnnData object with n_obs × n_vars = 12648 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS'
    var: 'gene_ids', 'protein?'
SNG confirmation via SNG- tag in BEST
View of AnnData object with n_obs × n_vars = 9458 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS', 'DOUBLETS'
    var: 'gene_ids', 'protein?'
Removing barcodes with no covariate data
View of AnnData object with n_obs × n_vars = 9458 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS', 'DOUBLETS'
    var: 'gene_ids', 'protein?'


/netapp/home/richardkperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/netapp/home/richardkperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/netapp/home/richardkperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/netapp/home/richardkperez/anaconda3/lib/python3.6/site-packages/ipykernel_launche

Structure details: AnnData object with n_obs × n_vars = 18406 × 32758 
    obs: 'SOURCE', 'STATUS', 'TR_UNT', 'batch', 'batch_cov', 'ind_cov', 'well'
    var: 'gene_ids-0', 'protein?-0', 'gene_ids-1', 'protein?-1'
Only considering the two last: ['.freemuxed', '.h5ad'].
Only considering the two last: ['.freemuxed', '.h5ad'].
Initial
AnnData object with n_obs × n_vars = 12635 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS'
    var: 'gene_ids', 'protein?'
SNG confirmation via SNG- tag in BEST
View of AnnData object with n_obs × n_vars = 7789 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS', 'DOUBLETS'
    var: 'gene_ids', 'protein?'
Removing barcodes with no covariate data
View of AnnData object with n_obs × n_vars = 7789 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS', 'DOUBLETS'
    var: 'gene_ids', 'protein?'
Structure details: AnnData object with n_obs × n

Initial
AnnData object with n_obs × n_vars = 23259 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS'
    var: 'gene_ids', 'protein?'
SNG confirmation via SNG- tag in BEST
View of AnnData object with n_obs × n_vars = 17449 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS', 'DOUBLETS'
    var: 'gene_ids', 'protein?'
Removing barcodes with no covariate data
View of AnnData object with n_obs × n_vars = 16725 × 32758 
    obs: 'N.SNP', 'BEST', 'mUMI', 'aUMI', 'NUM.SNPS', 'DROPLET.TYPE', 'BEST.GUESS', 'DOUBLETS'
    var: 'gene_ids', 'protein?'
Structure details: AnnData object with n_obs × n_vars = 110769 × 32758 
    obs: 'SOURCE', 'STATUS', 'TR_UNT', 'batch', 'batch_cov', 'ind_cov', 'well'
    var: 'gene_ids-0-0-0-0-0-0-0-0', 'protein?-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0-0', 'protein?-1-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0', 'protein?-1-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0', 'protein?-1-0-0-0-0-0', 

/netapp/home/richardkperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# House Keeping

In [4]:
%%capture
# Fix this mislabeling.
adata.obs.ind_cov = adata.obs.ind_cov.astype('object')
adata.obs.ind_cov = adata.obs.ind_cov.astype('str')
# Mislabeled by wrong excel label.
adata.obs.ind_cov[adata.obs.ind_cov=='101'] = '1240'
adata.obs.ind_cov[adata.obs.ind_cov=='101_101'] = '1240'
adata.obs.ind_cov[adata.obs.ind_cov=='0_900805200_900805200'] = '1804_1804'
adata.obs.ind_cov[adata.obs.ind_cov=='0_904194200_904194200'] = '904194200_904194200'
adata.obs.ind_cov[adata.obs.ind_cov=='0_1586_1586'] = '1586_1586'
adata.obs.ind_cov[adata.obs.ind_cov=='0_1763_1763'] = '1763_1763'
adata.obs.ind_cov[adata.obs.ind_cov=='1772'] = 'FLARE018'
adata.obs.ind_cov[adata.obs.ind_cov=='2132DNA01'] = 'FLARE019'
# Change to healthy to be more descriptive.
adata.obs['TR_UNT'][adata.obs['TR_UNT']=='-'] = 'HEALTHY'
# Remove '-' person...
adata = adata[adata.obs['ind_cov']!='-']
# 1333 is unknown. Remove.
adata = adata[adata.obs['ind_cov']!='1333_1333']
# 0_902320200_902320200 does not exist. Remove.
adata = adata[adata.obs['ind_cov']!='0_902320200_902320200']
print('Initial ind labels')
ind_list = np.unique(adata.obs['ind_cov'].tolist())
print(ind_list)
# Simplify labels
for ii in range(len(adata.obs['ind_cov'])):
    if adata.obs['ind_cov'][ii].startswith('0_'):
        adata.obs['ind_cov'][ii] = '_'.join(adata.obs['ind_cov'][ii].split('_')[1:])
print('Simplified labels')        
ind_list = np.unique(adata.obs['ind_cov'].tolist())
print(ind_list)
# Remove individuals with barcode counts less than 100
for ii in range(len(ind_list)):
    if np.sum(adata.obs['ind_cov']==ind_list[ii]) < 100:
        adata = adata[adata.obs['ind_cov']!=ind_list[ii]]
    else:
        continue
ind_list = np.unique(adata.obs['ind_cov'].tolist())

# From runs 1 and 2 these are labeled as healthy from CLUES but they are treated CLUES samples and not true healthies.
adata.obs['TR_UNT'] = adata.obs['TR_UNT'].astype('object')
adata.obs.loc[adata.obs.ind_cov == "1754_1754",'TR_UNT'] = "TR"
adata.obs.loc[adata.obs.ind_cov == "1771_1771",'TR_UNT'] = "TR"
adata.obs.loc[adata.obs.ind_cov == "1791_1791",'TR_UNT'] = "TR"
adata.obs.loc[adata.obs.ind_cov == "900805200_900805200",'TR_UNT'] = "TR"
adata.obs.loc[adata.obs.ind_cov == "1240",'TR_UNT'] = "TR"
adata.obs.loc[adata.obs.ind_cov == "1804_1804",'TR_UNT'] = "TR"

# Fix even more mislabeling ...
adata.obs['TR_UNT'] = adata.obs['TR_UNT'].astype('str')
# 904194200_904194200 #
adata.obs['TR_UNT'][(adata.obs['TR_UNT']=='Unknown (1 TR, 2 UNT)') & (adata.obs['ind_cov']=='904194200_904194200')] = 'UNT'
adata.obs['TR_UNT'][(adata.obs['TR_UNT']=='Unknown (2 TR, 1 UNT)') & (adata.obs['ind_cov']=='904194200_904194200')] = 'TR'
# 1586_1586 #
adata.obs['TR_UNT'][(adata.obs['TR_UNT']=='Unknown (1 TR, 2 UNT)') & (adata.obs['ind_cov']=='1586_1586')] = 'TR'
adata.obs['TR_UNT'][(adata.obs['TR_UNT']=='Unknown (2 TR, 1 UNT)') & (adata.obs['ind_cov']=='1586_1586')] = 'UNT'
# 1763_1763 # TR in Unknown (2 TR, 1 UNT)
adata.obs['TR_UNT'][(adata.obs['TR_UNT']=='Unknown (1 TR, 2 UNT)') & (adata.obs['ind_cov']=='1763_1763')] = 'UNT'
adata.obs['TR_UNT'][(adata.obs['TR_UNT']=='Unknown (2 TR, 1 UNT)') & (adata.obs['ind_cov']=='1763_1763')] = 'TR'

# Add female and broad covariates

In [5]:
%%capture
# preallocate space
adata.obs['Female'] = adata.obs.ind_cov.astype('object')
adata.obs['pop_cov'] = adata.obs.ind_cov.astype('object')
genderdict = {'1240':1, '1716_1716':1, '1804_1804':1,'1754_1754':1, '1731_1731':1, '1763_1763':1, 
'1771_1771':1, '1791_1791':1, '1892_1892':1,'904236200_904236200':1, '1243_1243':1, 'FLARE004':1, 'FLARE005':1, 'FLARE006':1, 'FLARE007':1,
 'FLARE009':1, 'FLARE011':1, 'FLARE013':1, '904194200_904194200':1, '1586_1586':1, '1763_1763':1,'FLARE014':1, 'FLARE015':1,
 'FLARE016':1, 'FLARE017':1, 'FLARE018':1, 'FLARE019':0, 'FLARE020':1,'FLARE2,3,or8':1, 'HC-002':0, 'HC-014':1,
 'HC-020':1, 'HC-501':1, 'HC-502':1, 'HC-503':1, 'HC-504':1, 'HC-506':1,
 'IGTB1290':1, 'IGTB1372':1, 'IGTB1506':1, 'IGTB1921':1, 'IGTB195':1, 'IGTB256':1, 'IGTB469':1,
 'IGTB508':1, 'IGTB514':1, 'IGTB826':1}

popdict = {'1240':'ASIAN', '1716_1716':'WHITE', '1804_1804':'ASIAN','1754_1754':'ASIAN', '1731_1731':'WHITE', '1763_1763':'ASIAN', 
'1771_1771':'ASIAN', '904194200_904194200':'WHITE', '1586_1586':"ASIAN", '1763_1763':'ASIAN' ,'1791_1791':'ASIAN', '1892_1892':'WHITE', '904236200_904236200':'ASIAN', '1243_1243':'WHITE', 
           'FLARE004':'ASIAN', 'FLARE005':'WHITE', 'FLARE006':'WHITE', 'FLARE007':'ASIAN',
 'FLARE009':'HISPANIC', 'FLARE011':'ASIAN', 'FLARE013':'BLACK', 'FLARE014':'BLACK', 'FLARE015':'ASIAN',
 'FLARE016':'ASIAN', 'FLARE017':'BLACK', 'FLARE018':'ASIAN', 'FLARE019':'HISPANIC', 'FLARE020':'WHITE','FLARE2,3,or8':'UNKNOWN', 'HC-002':'WHITE', 'HC-014':'ASIAN',
 'HC-020':'UNKNOWN', 'HC-501':'ASIAN', 'HC-502':'WHITE', 'HC-503':'WHITE', 'HC-504':'WHITE', 'HC-506':'WHITE',
 'IGTB1290':'WHITE', 'IGTB1372':'WHITE', 'IGTB1506':'WHITE', 'IGTB1921':'WHITE', 'IGTB195':'WHITE', 'IGTB256':'WHITE', 'IGTB469':'WHITE',
 'IGTB508':'WHITE', 'IGTB514':'WHITE', 'IGTB826':'WHITE'}

for key in genderdict.keys():
    adata.obs.Female[adata.obs.ind_cov==key] = genderdict[key]
    
for key in popdict.keys():
    adata.obs.pop_cov[adata.obs.ind_cov==key] = popdict[key]
    
# Preallocate space
adata.obs['Broad'] = adata.obs.SOURCE.astype('object')
# if immvar is source, assign 1.
adata.obs['Broad'][adata.obs.SOURCE == 'IMMVAR'] = 1
adata.obs['Broad'][adata.obs.SOURCE != 'IMMVAR'] = 0

adata.obs['Female'] = adata.obs['Female'].astype(dtype=np.float32)
adata.obs['Broad'] = adata.obs['Broad'].astype(dtype=np.float32)
adata.write(savepath, compression="gzip")

... storing 'SOURCE' as categorical
... storing 'STATUS' as categorical
... storing 'TR_UNT' as categorical
... storing 'batch_cov' as categorical
... storing 'ind_cov' as categorical
... storing 'well' as categorical
... storing 'pop_cov' as categorical
... storing 'protein?-0-0-0-0-0-0-0-0-0-0-0' as categorical
... storing 'protein?-1-0-0-0-0-0-0-0-0-0-0' as categorical
... storing 'protein?-1-0-0-0-0-0-0-0-0-0' as categorical
... storing 'protein?-1-0-0-0-0-0-0-0-0' as categorical
... storing 'protein?-1-0-0-0-0-0-0-0' as categorical
... storing 'protein?-1-0-0-0-0-0-0' as categorical
... storing 'protein?-1-0-0-0-0-0' as categorical
... storing 'protein?-1-0-0-0-0' as categorical
... storing 'protein?-1-0-0-0' as categorical
... storing 'protein?-1-0-0' as categorical
... storing 'protein?-1-0' as categorical
... storing 'protein?-1' as categorical


# Fix a bug and move Ab data out of gene matrix and take Center-log-ratio

In [6]:
from skbio.stats.composition import clr
adata = sc.read(savepath)
# Fix gene endings
genelist = adata.var_names.tolist()
genelist = [gn[:-2] for gn in genelist]
adata.var_names = genelist
# Find antibodies
ab_names = [ab for ab in genelist if 'pAbO' in ab]
# Find antibody index
ab_index = [genelist.index(ab) for ab in ab_names]
# Make new adata structure with just the antibody results
ABdata = adata[:, ab_index]
mat = csr_matrix(ABdata.X)
mat = mat.todense()
matrix = mat
print(str('abmatrix shape: ' + str(np.shape(matrix))))
# Make antibody into observations
for ii in range(len(ab_names)):
    LIST = np.asarray(matrix[:,ii])
    adata.obs[ab_names[ii]] = LIST
    
# Remove antibodies from gene list
abbool = [not ab for ab in adata.var_names.isin(ab_names)]
adata = adata[:, abbool]
print(str('Structure details: ' + str(adata)))

Only considering the two last: ['.flareAb', '.h5ad'].
Only considering the two last: ['.flareAb', '.h5ad'].
abmatrix shape: (156903, 20)
Structure details: View of AnnData object with n_obs × n_vars = 156903 × 32738 
    obs: 'SOURCE', 'STATUS', 'TR_UNT', 'batch', 'batch_cov', 'ind_cov', 'well', 'Female', 'pop_cov', 'Broad', 'CD38|CD38|j95-01|pAbO', 'CD95|FAS|j95-02|pAbO', 'CD28|CD28|j95-07|pAbO', 'CD27|CD27|j95-08|pAbO', 'CD197|CCR7|j95-09|pAbO', 'CD19|CD19|j95-10|pAbO', 'CD161|KLRB1|j95-11|pAbO', 'CD183|CXCR3|j95-12|pAbO', 'CD4|CD4|j95-14|pAbO', 'CD3|CD3E|j95-16|pAbO', 'CD196|CCR6|j95-17|pAbO', 'HLA-DR|CD74|j95-18|pAbO', 'CD45RO|PTPRC|j95-19|pAbO', 'CD14|CD14|j95-20|pAbO', 'CD194|CCR4|j95-21|pAbO', 'CD185|CXCR5|j95-22|pAbO', 'CD25|IL2RA|j95-24|pAbO', 'CD8|CD8A|j95-25|pAbO', 'CD127|IL7R|j95-27|pAbO', 'CD45RA|PTPRC|j95-28|pAbO'
    var: 'gene_ids-0-0-0-0-0-0-0-0-0-0-0', 'protein?-0-0-0-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0-0-0-0-0-0-0-0-0', 'protein?-1-0-0-0-0-0-0-0-0-0-0', 'gene_ids-1-0-0

# Remove all var lists to clean up structure

In [7]:
for key in list(adata.var.keys()):
    del adata.var[key]
adata.obs['disease_cov'] = adata.obs['TR_UNT']
del adata.obs['batch']
del adata.obs['STATUS']
del adata.obs['TR_UNT']

# Process flare cohort

In [8]:
import numpy as np
import scanpy.api as sc
from scipy.sparse import csr_matrix
import logging
import doubletdetection
import pandas as pd
import combat2
import combat
import patsy

##################
# Configure file #
##################
sc.settings.verbosity = 2
sc.settings.autoshow = False
logging.basicConfig(level=logging.INFO)
adata.obs['well'] = adata.obs['well'].astype('category')
adata.var_names_make_unique()
logging.info(str('Data structure details: ' + str(adata)))
mat = csr_matrix(adata.X)
logging.info(str('Data structure details: ' + str(adata)))
# Extract list of genes
genelist = adata.var_names.tolist()
# Find mitochondrial genes
mito_genes_names = [gn for gn in genelist if gn.startswith('MT-')]
logging.info(str('Mito genes: ' + str(mito_genes_names)))
# Find indices of mitochondrial genes
mito_genes = [genelist.index(gn) for gn in mito_genes_names]
# For each cell compute fraction of counts in mito genes vs. all genes
adata.obs['percent_mito'] = np.ravel(np.sum(adata[:, mito_genes].X, axis=1)) / np.ravel(np.sum(adata.X, axis=1))
# Add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = np.ravel(adata.X.sum(axis=1))
# Fix name to make it compatible with clinical variates
adata.obs['ind_cov'] = adata.obs['ind_cov'].astype('object')
adata.obs['ind_cov'][adata.obs['ind_cov'].isin(['1221'])] = '1221_1221'
adata.obs['ind_cov'][adata.obs['ind_cov'].isin(['1251'])] = '1251_1251'
adata.obs['ind_cov'][adata.obs['ind_cov'].isin(['1891'])] = '1891_1891'
indlist = np.unique(np.asarray(adata.obs['ind_cov'].tolist()))
# Get list of batches
batch_list = adata.obs['batch_cov'].tolist()
logging.info('Saving log(counts)+1 in .raw')
adata.raw = sc.pp.log1p(adata, copy=True)
logging.info('Filtering cells')
# Filter cells that have more than 10% of counts coming from mitochondrial genes.
adata = adata[adata.obs['percent_mito'] < 0.10]
logging.info(str('Data structure details: ' + str(adata)))
# Filter cells with abnormally low gene counts, high gene counts.
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_cells(adata, max_genes=2500)
sc.pp.filter_genes(adata, min_cells=100)
logging.info(str('Data structure details: ' + str(adata)))
logging.info('Saving raw counts')
adata.uns['barcodes'] = adata.obs_names.tolist()
adata.uns['genes'] = adata.var_names.tolist()
adata.uns['raw_counts'] = adata.X
logging.info('Normalizing total counts to 10,000')
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
logging.info('Log transforming data')
sc.pp.log1p(adata)
# Add platelet genes for purposes of regressing out their signature
adata.obs['PF4'] = adata.raw[:,"PF4"].X
adata.obs['SDPR'] = adata.raw[:,"SDPR"].X
adata.obs['GNG11'] = adata.raw[:,"GNG11"].X
adata.obs['PPBP'] = adata.raw[:,"PPBP"].X
logging.info('Making .obs into categories')
adata.strings_to_categoricals()
logging.info('Filtering genes')
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, inplace=True)
adata = adata[:, adata.var['highly_variable']]
logging.info(str('Data structure details: ' + str(adata)))
sc.pp.scale(adata, max_value=10)
# Compute PCA
sc.pp.pca(adata, random_state=1, svd_solver='arpack')
# Removing the two components most highly correlated with platelet counts
to_remove = pd.Series()
for ii in range(len(adata.obsm.X_pca[1,:])) :
    logging.info(str(np.corrcoef(adata.obsm.X_pca[:,ii], adata.obs["PF4"])[0,1]))
    if(np.absolute(np.corrcoef(adata.obsm.X_pca[:,ii], adata.obs["PF4"])[0,1]) > 0.1):
        adata.obs["PC"+str(ii)] = adata.obsm.X_pca[:,ii];
        to_remove = to_remove.append(pd.Series(["PC"+str(ii)]))
        logging.info(str(ii))
        logging.info(str(np.corrcoef(adata.obsm.X_pca[:,ii], adata.obs["PF4"])))

regressors = ['n_counts', 'percent_mito', 'Female']+to_remove.tolist();
logging.info('Regressing out total nUMIs, percentage mitochondrial UMIs, gender, platelet signature and pool')
logging.info('regressors: '+str(regressors))
sc.pp.regress_out(adata, regressors)
logging.info('Scaling expression data')
sc.pp.scale(adata, max_value=10)

# Combat adjustment                                                                                                                                                                                                           
logging.info('Running combat')
mod = patsy.dmatrix("~ disease_cov", adata.obs, return_type="dataframe")
mod = mod.reset_index(drop=True)
batch = adata.obs['batch_cov']
batch = batch.reset_index(drop=True)
adata.X = combat.combat(adata.X.transpose(), batch=batch, model=mod).transpose();

logging.info(str('Data structure details: ' + str(adata)))
logging.info('Removing samples that were processed twice...')
# Remove samples that were processed twice and get rid of their 1.10 run
remove = ['902289200_902289200', '1262_1262', '1270_1270', '1279_1279']
# Remove samples that were processed twice and get rid of their 8.9 run
# since the 8.9 run has the fewest number of cells, this adjustment is important for testing for cell proportion differences
# going forward, we will want to combine batches for the same individuals
remove2 = ['1472_1472', '1479_1479', '1480_1480', '1492_1492', '1522_1522', '1535_1535', '1602_1602', '1615_1615', '1621_1621', '1716_1716', '1726_1726', '1730_1730']
keep_indices = ~((adata.obs.ind_cov.isin(remove)) & (adata.obs.batch_cov == "lupus1.10"))
keep2_indices = ~((adata.obs.ind_cov.isin(remove2)) & (adata.obs.batch_cov == "lupus8.9"))
adata = adata[(keep_indices & keep2_indices)]
logging.info(str('Data structure details: ' + str(adata)))
# Unique list of individuals
people = np.unique(adata.obs['ind_cov'].values.tolist())
# Allocate space for total PMBCs per individual.
total_pbmcs = dict.fromkeys(people)
for p in people:
    total_pbmcs[p] = len(adata.obs_names[adata.obs['ind_cov'] == p])
adata.uns['total_pbmcs'] = total_pbmcs
logging.info('Saving processed data')
# Remove .var information we never use.
for key in list(adata.var.keys()):
    del adata.var[key]
logging.info(str('Structure details: ' + str(adata)))

# Switch name nomenclature to be consistent.
adata.obs['ind_cov'] = adata.obs['ind_cov'].astype('object')
adata.obs.ind_cov[adata.obs.ind_cov=='1586_1586'] = 'FLARE002'
adata.obs.ind_cov[adata.obs.ind_cov=='904194200_904194200'] = 'FLARE003'
adata.obs.ind_cov[adata.obs.ind_cov=='1763_1763'] = 'FLARE008'
adata.obs.ind_cov[adata.obs.ind_cov=='1754_1754'] = '1754'
adata.obs.ind_cov[adata.obs.ind_cov=='1771_1771'] = '1771'
adata.obs.ind_cov[adata.obs.ind_cov=='1791_1791'] = '1791'
adata.obs.ind_cov[adata.obs.ind_cov=='1804_1804'] = '1804'

# Swith disease covariate convention from healthy, TR, UNT, to Healty, Managed, TR, UNT
# Cross-sectional individuals
CrossX = ['1240', '1754', '1771', '1791', '1804']
adata.obs['disease_cov'] = adata.obs['disease_cov'].astype('object')
adata.obs['disease_cov'][adata.obs['ind_cov'].isin(CrossX)] = 'Managed'
adata.obs['disease_cov'][adata.obs['disease_cov']=='HEALTHY'] = 'Healthy'
adata.obs['disease_cov'][adata.obs['disease_cov']=='TR'] = 'Treated'
adata.obs['disease_cov'][adata.obs['disease_cov']=='UNT'] = 'Flare'

# Add follow up therapy covariates to adata
# No therapy data for 1791, FLARE018
# Indicate steroid therapy.
steroid = np.zeros(len(adata.obs_names))
steroidpt = ['FLARE008', '1771','1804', 'FLARE003', 'FLARE004', 'FLARE006',
 'FLARE007', 'FLARE009', 'FLARE011', 'FLARE013', 'FLARE015', 'FLARE017', 'FLARE020']
steroid[adata.obs['ind_cov'].isin(steroidpt)] = 1
# Indicate plaquenil therapy.
plaquenil = np.zeros(len(adata.obs_names))
plaquenilpt = ['1240', '1754', 'FLARE008', '1771', 'FLARE003','FLARE005', 'FLARE014', 
               'FLARE009', 'FLARE011', 'FLARE017', 'FLARE020']
plaquenil[adata.obs['ind_cov'].isin(plaquenilpt)] = 1
# Indicate cellcept therapy.
cellcept = np.zeros(len(adata.obs_names))
cellceptpt = ['1240', '1771', 'FLARE004', 'FLARE005', 'FLARE020']
cellcept[adata.obs['ind_cov'].isin(cellceptpt)] = 1
# Indicate rituxumab therapy.
rituxumab = np.zeros(len(adata.obs_names))
rituxumabpt = ['FLARE013', 'FLARE003', 'FLARE002']
rituxumab[adata.obs['ind_cov'].isin(rituxumabpt)] = 1
# Indicate cyclophosphamide therapy.
cyclophosphamide = np.zeros(len(adata.obs_names))
cyclophosphamidept = ['FLARE011', 'FLARE013', 'FLARE015', 'FLARE019']
cyclophosphamide[adata.obs['ind_cov'].isin(cyclophosphamidept)] = 1
adata.obs['steroid'] = steroid
adata.obs['plaquenil'] = plaquenil
adata.obs['cellcept'] = cellcept
adata.obs['cyclophosphamide'] = cyclophosphamide
adata.obs['rituxumab'] = rituxumab
# Save both to active analysis file and also no_norm file for publication.
adata.write(saveno_norm_path, compression="gzip")
adata.write(savepath, compression="gzip")

INFO:root:Data structure details: AnnData object with n_obs × n_vars = 156903 × 32738 
    obs: 'SOURCE', 'batch_cov', 'ind_cov', 'well', 'Female', 'pop_cov', 'Broad', 'CD38|CD38|j95-01|pAbO', 'CD95|FAS|j95-02|pAbO', 'CD28|CD28|j95-07|pAbO', 'CD27|CD27|j95-08|pAbO', 'CD197|CCR7|j95-09|pAbO', 'CD19|CD19|j95-10|pAbO', 'CD161|KLRB1|j95-11|pAbO', 'CD183|CXCR3|j95-12|pAbO', 'CD4|CD4|j95-14|pAbO', 'CD3|CD3E|j95-16|pAbO', 'CD196|CCR6|j95-17|pAbO', 'HLA-DR|CD74|j95-18|pAbO', 'CD45RO|PTPRC|j95-19|pAbO', 'CD14|CD14|j95-20|pAbO', 'CD194|CCR4|j95-21|pAbO', 'CD185|CXCR5|j95-22|pAbO', 'CD25|IL2RA|j95-24|pAbO', 'CD8|CD8A|j95-25|pAbO', 'CD127|IL7R|j95-27|pAbO', 'CD45RA|PTPRC|j95-28|pAbO', 'disease_cov'
INFO:root:Data structure details: AnnData object with n_obs × n_vars = 156903 × 32738 
    obs: 'SOURCE', 'batch_cov', 'ind_cov', 'well', 'Female', 'pop_cov', 'Broad', 'CD38|CD38|j95-01|pAbO', 'CD95|FAS|j95-02|pAbO', 'CD28|CD28|j95-07|pAbO', 'CD27|CD27|j95-08|pAbO', 'CD197|CCR7|j95-09|pAbO', 'CD19|CD19|

filtered out 865 cells that have more than  2500 genes expressed
filtered out 17883 genes that are detected in less than 100 cells


INFO:root:Data structure details: AnnData object with n_obs × n_vars = 153955 × 14855 
    obs: 'SOURCE', 'batch_cov', 'ind_cov', 'well', 'Female', 'pop_cov', 'Broad', 'CD38|CD38|j95-01|pAbO', 'CD95|FAS|j95-02|pAbO', 'CD28|CD28|j95-07|pAbO', 'CD27|CD27|j95-08|pAbO', 'CD197|CCR7|j95-09|pAbO', 'CD19|CD19|j95-10|pAbO', 'CD161|KLRB1|j95-11|pAbO', 'CD183|CXCR3|j95-12|pAbO', 'CD4|CD4|j95-14|pAbO', 'CD3|CD3E|j95-16|pAbO', 'CD196|CCR6|j95-17|pAbO', 'HLA-DR|CD74|j95-18|pAbO', 'CD45RO|PTPRC|j95-19|pAbO', 'CD14|CD14|j95-20|pAbO', 'CD194|CCR4|j95-21|pAbO', 'CD185|CXCR5|j95-22|pAbO', 'CD25|IL2RA|j95-24|pAbO', 'CD8|CD8A|j95-25|pAbO', 'CD127|IL7R|j95-27|pAbO', 'CD45RA|PTPRC|j95-28|pAbO', 'disease_cov', 'percent_mito', 'n_counts', 'n_genes'
    var: 'n_cells'
INFO:root:Saving raw counts
INFO:root:Normalizing total counts to 10,000
INFO:root:Log transforming data
INFO:root:Making .obs into categories
... storing 'ind_cov' as categorical
INFO:root:Filtering genes
INFO:root:Data structure details: View o

computing PCA with n_comps = 50
computing PCA on highly variable genes


INFO:root:0.03289466194222416
INFO:root:-0.024256113962236408
INFO:root:0.04943114416692915
INFO:root:0.6864686745989058
INFO:root:3
INFO:root:[[1.         0.68646867]
 [0.68646867 1.        ]]
INFO:root:-0.019245743459576612
INFO:root:-0.07126194870209629
INFO:root:-0.021853658160646584
INFO:root:-0.028917875408797835
INFO:root:-0.019351945929253878
INFO:root:-0.013843134135526694
INFO:root:-0.017616936052614027
INFO:root:-0.010341023437602485
INFO:root:-0.022153444329794957
INFO:root:-0.03416881233664512
INFO:root:0.021039658915079317
INFO:root:0.0014076111880392391
INFO:root:-0.21526329380911205
INFO:root:16
INFO:root:[[ 1.         -0.21526329]
 [-0.21526329  1.        ]]
INFO:root:0.02776355131900495
INFO:root:0.012410002207283118
INFO:root:0.012528772553391196
INFO:root:0.023349977152981594
INFO:root:-0.018581663064055225
INFO:root:-0.006097165326742777
INFO:root:0.006777747781682785
INFO:root:-0.023183382807453815
INFO:root:-0.02214776765600093
INFO:root:-0.010306573984465384
INF

...

INFO:root:Scaling expression data


    finished (0:02:15.59)


INFO:root:Running combat
found 4 batches
found 0 numerical covariates...
found 2 categorical variables:	disease_cov[T.TR], disease_cov[T.UNT]
Standardizing Data across genes.
Fitting L/S model and finding priors
Finding parametric adjustments


Adjusting data


INFO:root:Data structure details: AnnData object with n_obs × n_vars = 153955 × 1164 
    obs: 'SOURCE', 'batch_cov', 'ind_cov', 'well', 'Female', 'pop_cov', 'Broad', 'CD38|CD38|j95-01|pAbO', 'CD95|FAS|j95-02|pAbO', 'CD28|CD28|j95-07|pAbO', 'CD27|CD27|j95-08|pAbO', 'CD197|CCR7|j95-09|pAbO', 'CD19|CD19|j95-10|pAbO', 'CD161|KLRB1|j95-11|pAbO', 'CD183|CXCR3|j95-12|pAbO', 'CD4|CD4|j95-14|pAbO', 'CD3|CD3E|j95-16|pAbO', 'CD196|CCR6|j95-17|pAbO', 'HLA-DR|CD74|j95-18|pAbO', 'CD45RO|PTPRC|j95-19|pAbO', 'CD14|CD14|j95-20|pAbO', 'CD194|CCR4|j95-21|pAbO', 'CD185|CXCR5|j95-22|pAbO', 'CD25|IL2RA|j95-24|pAbO', 'CD8|CD8A|j95-25|pAbO', 'CD127|IL7R|j95-27|pAbO', 'CD45RA|PTPRC|j95-28|pAbO', 'disease_cov', 'percent_mito', 'n_counts', 'n_genes', 'PF4', 'SDPR', 'GNG11', 'PPBP', 'PC3', 'PC16'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'barcodes', 'genes', 'raw_counts', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'
INFO:root:Removing samples that were processed twice.

# Run basic analysis

In [9]:
#######################
# Louvain and friends #
#######################
# Set parameters
intialization = 1
n_components = 20
resolution = 3
# Run louvain clustering on theoretical future gene expression per cell
logging.info('Estimating louvain cluster identities for gene expression values.')
sc.pp.pca(adata, random_state=intialization, svd_solver='arpack')
logging.info('PCA complete.')
sc.pp.neighbors(adata, random_state=intialization)
logging.info('KNN complete.')
sc.tl.diffmap(adata)
logging.info('diffmap complete.')
sc.tl.louvain(adata, random_state=15, resolution=resolution)
sc.tl.leiden(adata, random_state=intialization, resolution=resolution)
logging.info('Louvain complete.')
sc.tl.umap(adata, random_state=intialization)
logging.info('UMAP complete.')
logging.info('Making .obs into categories')
adata.strings_to_categoricals()
adata.write(savepath, compression="gzip")
logging.info('Basic analysis complete.')

INFO:root:Estimating louvain cluster identities for gene expression values.


computing PCA with n_comps = 50


INFO:root:PCA complete.


    finished (0:00:24.25)
computing neighbors
    using 'X_pca' with n_pcs = 50


INFO:root:KNN complete.


...

INFO:root:Louvain complete.


computing UMAP
    using 'X_pca' with n_pcs = 50


INFO:root:UMAP complete.
INFO:root:Making .obs into categories


    finished (0:03:58.60)


INFO:root:Basic analysis complete.
